First install the following packages:

- `requests`
- `tqdm`
- `xarray-leaflet`

The recommended way is: `conda install -c conda-forge requests tqdm xarray_leaflet`.

We start by importing the modules that we will need:

In [ ]:
import requests
import os
from tqdm import tqdm
import numpy as np
import zipfile
import rioxarray
import xarray_leaflet
from xarray_leaflet.transform import passthrough
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from ipyleaflet import Map, basemaps, projections
from ipywidgets import Output

We will display the DEM (digital elevation model) for Europe from the [HydroSHEDS](https://hydrosheds.org) dataset, which represents the terrain. Let's first download the data:

In [ ]:
url = 'https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/hydrosheds/sa_30s_zip_grid/eu_dem_30s_grid.zip'
filename = os.path.basename(url)
name = filename[:filename.find('_grid')]
adffile = name + '/' + name + '/w001001.adf'

if not os.path.exists(adffile):
    r = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        total_length = int(r.headers.get('content-length'))
        for chunk in tqdm(r.iter_content(chunk_size=1024), total=(total_length/1024) + 1):
            if chunk:
                f.write(chunk)
                f.flush()
    zip = zipfile.ZipFile(filename)
    zip.extractall('.')

In [ ]:
da = rioxarray.open_rasterio(adffile, masked=True)
da = da.sel(band=1)

We define a custom color map that will allow us to show the elevation only from a given value, so that we can still see the basemap for low elevation values.

In [ ]:
cmap = plt.cm.get_cmap('jet', 256)
newcolors = cmap(np.linspace(0, 1, 256))
newcolors[:,-1][0:30] = 0  # transparent for low values
cmap = ListedColormap(newcolors)

We will show a map with a projection centered on the north pole. This demonstrates the ability for xarray-leaflet to reproject.

In [ ]:
m = Map(center=(64.47601699176504, -40.652543092440325),
        zoom=1,
        basemap=basemaps.NASAGIBS.BlueMarble3413,
        crs=projections.EPSG3413,
        interpolation='nearest')
m

In [ ]:
debug_output = Output()
debug_output

In [ ]:
l = da.leaflet.plot(m, transform2=passthrough, colormap=cmap, fit_bounds=False, debug_output=debug_output)
l.interact(opacity=(0.0,1.0,0.1))